In [1]:
import pickle
import pandas as pd

#scr
import sys
sys.path.append("../")
import src.support as sp

# data input

In [2]:
df = pd.read_csv("../data/test.csv")
df.head(2)

,id,carat,cut,color,clarity,depth,table,x,y,z
0,0,0.32,Ideal,I,SI1,60.5,58.0,4.43,4.49,2.70
1,1,1.24,Premium,I,SI1,62.9,60.0,6.80,6.74,4.26


quitamos las columnas para que este igual el input

In [3]:
df.drop(['table', 'depth', 'x', 'y', 'z'], axis = 1, inplace = True)
df.head(1)

,id,carat,cut,color,clarity
0,0,0.32,Ideal,I,SI1


# transformers input

cargamos el mejor modelo. en este caso, será el gradient boosting:

In [4]:
encoding_clarity, encoding_color, encoding_cut, estandarizacion, modelo = sp.transformers_input('encoding_clarity', 'encoding_color', 'encoding_cut', 'estandarizacion', 'modelo_GradientBoosting')

In [5]:
# estandarizamos todas las columnas que son de tipo numérico usando el transformer que nos guardamos cuando hicimos el modelo

estandarizadas = pd.DataFrame(estandarizacion.transform(df[["carat"]]), columns = ["carat"])
#estandarizadas.drop('price', axis = 1, inplace = True)
estandarizadas.head()

,carat
0,-0.593750
1,0.843750
2,1.500000
3,0.078125
4,1.250000


In [6]:
# hacemos el encoding del resto de las variables

clarity = pd.DataFrame(encoding_clarity.transform(df[["clarity"]]), columns= ["clarity"])
cut = pd.DataFrame(encoding_cut.transform(df[["cut"]]), columns= ["cut"])
color = pd.DataFrame(encoding_color.transform(df[["color"]]), columns= ["color"])

In [7]:
df.head(1)

,id,carat,cut,color,clarity
0,0,0.32,Ideal,I,SI1


In [8]:
df_final = pd.concat([estandarizadas[['carat']], cut, color, clarity, #estandarizadas[['x', 'y', 'z']]
                      ]
                     , axis = 1)
df_final

,carat,cut,color,clarity
0,-0.593750,4,1,4
1,0.843750,3,1,4
2,1.500000,3,6,4
3,0.078125,3,6,3
4,1.250000,0,5,3
...,...,...,...,...
13480,0.625000,3,3,4
13481,0.312500,2,6,4
13482,-0.625000,4,4,5
13483,0.859375,4,1,4


In [9]:
# hacemos la predicción para nuestro nuevo paciente
price = modelo.predict(df_final)
price

array([6.13334486, 8.61874771, 9.46296938, ..., 6.48337898, 8.66841503,
       7.96615174])

In [10]:
df['price'] = price

In [11]:
df[['id', 'price']].to_csv('../data/submission06.csv', index = False)